In [12]:
import random

GRID_SIZE = 5
actions = ['up', 'down', 'left', 'right', 'pickup', 'dropoff']
pickup_location = (0, 0)
dropoff_location = (4, 4)
states = [(x, y, has_passenger) for x in range(GRID_SIZE) for y in range(GRID_SIZE) for has_passenger in [False, True]]

In [14]:
def transition_model(state, action):
    x, y, has_passenger = state
    if action in ['pickup', 'dropoff']:
        if action == 'pickup' and not has_passenger and (x, y) == pickup_location:
            return [(1.0, (x, y, True))]
        elif action == 'dropoff' and has_passenger and (x, y) == dropoff_location:
            return [(1.0, (x, y, False))]
        else:
            return [(1.0, state)]
    else:
        new_x, new_y = x, y
        if action == 'up' and x > 0:
            new_x -= 1
        elif action == 'down' and x < GRID_SIZE - 1:
            new_x += 1
        elif action == 'left' and y > 0:
            new_y -= 1
        elif action == 'right' and y < GRID_SIZE - 1:
            new_y += 1
        return [(1.0, (new_x, new_y, has_passenger))]

In [16]:
def rewards(s, a, s_):
    x, y, has_passenger = s
    x_, y_, has_passenger_ = s_
    if a == 'pickup':
        if (x, y) == pickup_location and not has_passenger:
            return -1
        else:
            return -10
    elif a == 'dropoff':
        if has_passenger and (x, y) == dropoff_location:
            return 20
        else:
            return -10
    else:
        return -1

In [18]:
def value_iteration(states, actions, transition_model, rewards, gamma, theta):
    V = {s: 0 for s in states}
    while True:
        delta = 0
        for s in states:
            v = V[s]
            V[s] = max(
                sum(p * (rewards(s, a, s_) + gamma * V[s_])
                    for p, s_ in transition_model(s, a))
                for a in actions
            )
            delta = max(delta, abs(v - V[s]))
        if delta < theta:
            break
    policy = {
        s: max(actions, key=lambda a: sum(p * (rewards(s, a, s_) + gamma * V[s_])
                                          for p, s_ in transition_model(s, a)))
        for s in states
    }
    return V, policy

In [20]:
def policy_iteration(states, actions, transition_model, rewards, gamma, theta):
    policy = {s: random.choice(actions) for s in states}
    V = {s: 0 for s in states}
    is_policy_stable = False
    while not is_policy_stable:
        while True:
            delta = 0
            for s in states:
                v = V[s]
                a = policy[s]
                V[s] = sum(p * (rewards(s, a, s_) + gamma * V[s_])
                           for p, s_ in transition_model(s, a))
                delta = max(delta, abs(v - V[s]))
            if delta < theta:
                break
        is_policy_stable = True
        for s in states:
            old_action = policy[s]
            new_action = max(actions, key=lambda a: sum(
                p * (rewards(s, a, s_) + gamma * V[s_]) for p, s_ in transition_model(s, a)))
            if old_action != new_action:
                is_policy_stable = False
            policy[s] = new_action
    return V, policy

In [22]:
V, policy = value_iteration(states, actions, transition_model, rewards, gamma=0.9, theta=1e-4)

In [24]:
print("Value Iteration Policy (without passenger):")
for x in range(GRID_SIZE):
    for y in range(GRID_SIZE):
        print(f"({x},{y}): {policy[(x, y, False)]}", end='\t')
    print()

Value Iteration Policy (without passenger):
(0,0): pickup	(0,1): left	(0,2): left	(0,3): left	(0,4): left	
(1,0): up	(1,1): up	(1,2): up	(1,3): up	(1,4): up	
(2,0): up	(2,1): up	(2,2): up	(2,3): up	(2,4): up	
(3,0): up	(3,1): up	(3,2): up	(3,3): up	(3,4): up	
(4,0): up	(4,1): up	(4,2): up	(4,3): up	(4,4): up	


In [26]:
print("\nValue Iteration Policy (with passenger):")
for x in range(GRID_SIZE):
    for y in range(GRID_SIZE):
        print(f"({x},{y}): {policy[(x, y, True)]}", end='\t')
    print()


Value Iteration Policy (with passenger):
(0,0): down	(0,1): down	(0,2): down	(0,3): down	(0,4): down	
(1,0): down	(1,1): down	(1,2): down	(1,3): down	(1,4): down	
(2,0): down	(2,1): down	(2,2): down	(2,3): down	(2,4): down	
(3,0): down	(3,1): down	(3,2): down	(3,3): down	(3,4): down	
(4,0): right	(4,1): right	(4,2): right	(4,3): right	(4,4): dropoff	


In [28]:
V, policy = policy_iteration(states, actions, transition_model, rewards, gamma=0.9, theta=1e-4)

In [30]:
print("\nPolicy Iteration Policy (without passenger):")
for x in range(GRID_SIZE):
    for y in range(GRID_SIZE):
        print(f"({x},{y}): {policy[(x, y, False)]}", end='\t')
    print()


Policy Iteration Policy (without passenger):
(0,0): pickup	(0,1): left	(0,2): left	(0,3): left	(0,4): left	
(1,0): up	(1,1): up	(1,2): up	(1,3): up	(1,4): up	
(2,0): up	(2,1): up	(2,2): up	(2,3): up	(2,4): up	
(3,0): up	(3,1): up	(3,2): up	(3,3): up	(3,4): up	
(4,0): up	(4,1): up	(4,2): up	(4,3): up	(4,4): up	


In [32]:
print("\nPolicy Iteration Policy (with passenger):")
for x in range(GRID_SIZE):
    for y in range(GRID_SIZE):
        print(f"({x},{y}): {policy[(x, y, True)]}", end='\t')
    print()


Policy Iteration Policy (with passenger):
(0,0): down	(0,1): down	(0,2): down	(0,3): down	(0,4): down	
(1,0): down	(1,1): down	(1,2): down	(1,3): down	(1,4): down	
(2,0): down	(2,1): down	(2,2): down	(2,3): down	(2,4): down	
(3,0): down	(3,1): down	(3,2): down	(3,3): down	(3,4): down	
(4,0): right	(4,1): right	(4,2): right	(4,3): right	(4,4): dropoff	
